## Installation

In [ ]:
# 
# !pip install huggingface_hub

In [ ]:
# 
# !pip install --upgrade transformers

In [ ]:
#
# from huggingface_hub import notebook_login
# notebook_login()

In [1]:
# 
!pip install -U vllm torch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 394.6/394.6 MB 4.3 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 865.2/865.2 MB 1.5 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.0/117.0 kB 52.6 kB/s eta 0:00:00ta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 111.0/111.0 kB 116.0 kB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 13.0 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 393.1/393.1 MB 4.3 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 101.0 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 63.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.7/897.7 kB 50.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 571.0/571.0 MB 2.8 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.2/200.2 MB 5.6 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━

# vllm server running in python subprocess

In [20]:
# 
import subprocess
# import atexit
import os

cmd = [
    "vllm", "serve", "Menlo/Jan-nano-128k",
    "--host", "0.0.0.0",
    "--port", "1234",
    "--enable-auto-tool-choice",
    "--tool-call-parser", "hermes",
    "--rope-scaling", '{"rope_type":"yarn","factor":3.2,"original_max_position_embeddings":40960}',
    "--max-model-len", "32768",
    "--tensor-parallel-size", "2",
    "--gpu-memory-utilization", "0.8"
]

# cmd = [
#       "vllm" ,"serve", "google/gemma-3n-e4b-it" ,
#       "--host" ,"0.0.0.0" ,
#       "--port","1234" ,
#       "--tensor-parallel-size", "2" ,
#       "--dtype" ,"half",
#       "--max-model-len" ,"32768" ,
#       "--gpu-memory-utilization","0.95"

# ]



process = subprocess.Popen(
    cmd,
    stdout=open("jan128k.log", "w"),
    stderr=subprocess.STDOUT
)

print(f"Started vLLM with PID {process.pid}")


Started vLLM with PID 924


### vllm logs

In [22]:
# 
!ps -ax | grep vllm
!tail -n 200 jan128k.log

    924 ?        Sl     0:21 /usr/bin/python3 /usr/local/bin/vllm serve Menlo/Jan-nano-128k --host 0
   1097 pts/0    Ss+    0:00 /bin/bash -c ps -ax | grep vllm
   1099 pts/0    S+     0:00 grep vllm
INFO 07-02 06:26:36 [__init__.py:244] Automatically detected platform cuda.
2025-07-02 06:26:37.469266: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1751437597.493021     924 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1751437597.500211     924 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
INFO 07-02 06:26:48 [api_server.py:1287] vLLM API server version 0.9.1
INFO 07-02 06:26:50 [cli_args.py:309] non-default args: {'host': '0.0.0.0', 'port': 1234,

In [23]:
# 
# Check logs
# import time
# time.sleep(30)  # Wait for startup
# !tail -n 50 jan128k.log

# Verify server is running
!ps -ax | grep vllm
!netstat -tulnp | grep 1234

    924 ?        Sl     0:21 /usr/bin/python3 /usr/local/bin/vllm serve Menlo/Jan-nano-128k --host 0
   1101 pts/0    Ss+    0:00 /bin/bash -c ps -ax | grep vllm
   1103 pts/0    S+     0:00 grep vllm
tcp        0      0 0.0.0.0:1234            0.0.0.0:*               LISTEN      924/python3         


## Ngrok setup

In [24]:
# 
# Ngrok setup
!pip install pyngrok --quiet


In [25]:
# 
from pyngrok import ngrok

# Replace with your actual ngrok authtoken
ngrok.set_auth_token("2rqzh1D81plieczVNZ0A2g6Xzrh_2aVgQvijc8brnWUaHd8AF")  # 🛡️ Required for authenticated access :contentReference[oaicite:3]{index=3}

tunnel = ngrok.connect(1234)
print("Public URL:", tunnel.public_url)

Public URL: https://4389-34-10-227-245.ngrok-free.app


In [26]:
import requests
import json

try:
    response = requests.get(f"{tunnel.public_url}/health", timeout=10)
    print("Server health:", response.status_code)
except Exception as e:
    print("Server not ready:", e)

Server health: 200


### List and Kill ngrok

In [27]:
!ps -ax | grep ngrok
!killall ngrok

    891 ?        Sl     0:05 /root/.config/ngrok/ngrok start --none --log stdout
   1124 pts/0    Ss+    0:00 /bin/bash -c ps -ax | grep ngrok
   1126 pts/0    S+     0:00 grep ngrok


## Tunnelmole

In [28]:
# 
!curl -O https://install.tunnelmole.com/t357g/install && sudo bash install


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   721    0   721    0     0   3130      0 --:--:-- --:--:-- --:--:--  3134
Installing Tunnelmole for Linux
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  1427  100  1427    0     0   3249      0 --:--:-- --:--:-- --:--:--  3243
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 17.4M  100 17.4M    0     0  19.2M      0 --:--:-- --:--:-- --:--:-- 19.2M
  _______                     _                 _      
 |__   __|                   | |               | |     
    | |_   _ _ __  _ __   ___| |_ __ ___   ___ | | ___ 
    | | | | | '_ \| '_ \ / _ \ | '_ ` _ \ / _ \| |/ _ \ 
    | | |_| | | | | | | |  __/ |

In [29]:
import subprocess

process = subprocess.Popen(
    ["tmole", "1234"],
    stdout=open("tunnelmole.log", "w"),
    stderr=subprocess.STDOUT
)
print("Tunnelmole started with PID:", process.pid)

Tunnelmole started with PID: 1147


In [30]:
!tail -n 20 tunnelmole.log

https://f8kfix-ip-34-10-227-245.tunnelmole.net ⟶   http://localhost:1234
http://f8kfix-ip-34-10-227-245.tunnelmole.net ⟶   http://localhost:1234


Share to Reddit: https://dashboard.tunnelmole.com/share/reddit/aHR0cHM6Ly9mOGtmaXgtaXAtMzQtMTAtMjI3LTI0NS50dW5uZWxtb2xlLm5ldA==
Share to X/Twitter: https://dashboard.tunnelmole.com/share/twitter/aHR0cHM6Ly9mOGtmaXgtaXAtMzQtMTAtMjI3LTI0NS50dW5uZWxtb2xlLm5ldA==
Share to Hacker News: https://dashboard.tunnelmole.com/share/hackernews/aHR0cHM6Ly9mOGtmaXgtaXAtMzQtMTAtMjI3LTI0NS50dW5uZWxtb2xlLm5ldA==






Hey there 👋. I'm Robbie, the creator of Tunnelmole. I'm looking to improve Tunnelmole and your feedback could make a huge difference!

Would you be interested in participating in a short interview over video call? As thanks, I'm offering a free 3-month Tunnelmole subscription. Send me an email at robbie-cahill@proton.me and we can set something up.

Thanks for supporting the community!


